In [18]:
import openpyxl as xl
from decimal import Decimal, getcontext

class Result:
    getcontext().prec = 9
    target_cols = [4,11,12,22]

    def __init__(self, n_of_digits):
        '''self.n_of_digits - кол-во знаков порсле запятой'''
        self.n_of_digits = n_of_digits 
        self.result = []
        
    def load_prepare_make(self, link):
        '''Загружает Excel-таблицу, обрабатывает её, полноценно создает атрибут result'''
        wb = xl.load_workbook(f'таблицы/{link}.xlsx', data_only=True)
        sheet = wb.active
        self.__total_gos_price = Decimal(sheet.cell(row=6,column=7).value)
        for i in range(24,sheet.max_row):
            fin_row = []
            for j in Result.target_cols:
                if j == 4:
                    fin_row.append(str(Result.first_col_creator(sheet.cell(row=i,column=j).value)))
                else:
                    fin_row.append(str(sheet.cell(row=i,column=j).value))
            self.result.append(fin_row)
        for i in self.result:
            i.append(Decimal(i[1]) * Decimal(i[2]))
        for i in self.result:
            i.append(Decimal(i[3]) / Decimal(i[2]) - 1)   

    #################################################################

    @staticmethod
    def first_col_creator(txt):
        '''Метод формирует первый элемент каждой строки списка .result'''
        txt = txt.lower().split()
        ml_num = ['мл', '№']
        first_col = []
        for i in txt:
            for j in ml_num:
                if j in i:
                    first_col.append(i.replace(j,''))
        first_col = [int(i) for i in first_col]
        res = 1
        for i in first_col:
            res *= i
        return res

    #################################################################

    def defining_consts(self):
        '''Определение общей суммы и суммы строки'''
        self.our_total = sum([i[4] for i in self.result])
        self.rows_sum = [i[4] for i in self.result]
        self.difference = self.__total_gos_price - self.our_total

    def update(self):
        '''Обновляет значения суммы строки и процента до предела реестра'''
        for i in range(len(self.result)): #сумма строки
            self.result[i][4] = Decimal(self.result[i][1]) * Decimal(self.result[i][2])
            self.result[i][5] = (Decimal(self.result[i][3]) / Decimal(self.result[i][2])) - 1
        self.defining_consts()

    #################################################################

    def count_up_to_n_decimal_places(self,index):
        '''Подбивает строку под n-знаков'''
        price_of_ml = Decimal(self.result[index][2]) / Decimal(self.result[index][0])
        price_of_ml = str(price_of_ml).split('.')[0] + '.' + str(price_of_ml).split('.')[1][:self.n_of_digits]
        new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        while new_full_price != Decimal(str(new_full_price).split('.')[0] + '.' + str(new_full_price).split('.')[1][:2]) and Decimal(price_of_ml) > 0:
            price_of_ml = str(Decimal(price_of_ml) -Decimal(str(1 / ( 10 ** self.n_of_digits))))
            new_full_price = Decimal(price_of_ml) * Decimal(self.result[index][0])
        self.result[index][2] = str(new_full_price)
        self.update()

    def minimize_diff(self):
        pass

    def divide_to_2_rows(self, index):
        index -= 1
        summ, amount1, NxV = Decimal(self.result[index][4]), self.result[index][1], self.result[index][0]
        price_ml_1 = Decimal(input(f'Цена за мл: {summ / Decimal(amount1) / Decimal(NxV)}\nЦена за мл 1: '))
        price_ml_2 = Decimal(input(f'Цена за мл: {summ / Decimal(amount1) / Decimal(NxV)}\nЦена за мл 2: '))
        lst=[]
        # for i in range(1, int(amount1) + 1):
        #     amount2 = int(amount1) - i
        #     lst[i] = 
        lst2 = [(summ - ((Decimal(amount1) * Decimal(NxV) * Decimal(price_ml_1)) + (Decimal(int(amount1) - i) * Decimal(NxV) * Decimal(price_ml_2)))) for i in range(1,int(amount1))]
        min_diff, min_index = min(lst2), lst.index(min(lst2))
        print(f'Остаток: {min_diff} | Кол-во упаковок по цене {price_ml_1} за мл - {min_index} | Кол-во упаковок по цене {price_ml_2} за мл - {self.result[index][1] - min_index}')
    
    #################################################################
    
    def price_price_output(self):
        for j, i in enumerate(self.result, start=1):
            price, const = Decimal(i[2]), Decimal(i[0])
            print(f'{j}) {price}---{price / const}')

    def final_print(self):
        '''Метод финальной печати .result'''
        self.update()
        for i in self.result:
            print(str(i))
        print('Цена госзакупки: ' + str(self.__total_gos_price))
        print('Цена контракта:  ' + str(self.our_total))
        print('Разница:         ' + str(self.difference))
        print()


In [ ]:
# Result(n) n - кол-во знаков после запятой 
# load_prepare_make('name') name - имя файла без .xlsx
# count_up_to_n_decimal_places(index) index - номер строки, которую надо просчитать 
# update() пересчитывает все суммы, это делается автоматически, но для безопасности надо пересчитывать еще раз
# final_print() вывод таблицы и всех сумм
# price_price_output() выводит в формате    цена за упаковку --- цена за миллилитр

In [19]:
res = Result(8) # в скобках количество знаков после запятой в цене за миллилитр
res.load_prepare_make('0271')  # в кавыках имя файла без .xlsx
res.update() 
for i in range(len(res.result)):
    res.count_up_to_n_decimal_places(i)  # просчет каждой строки по индексу под n знаков после запятой в цене за мл
res.update()
res.final_print()  # функция финального вывода таблицы
res.price_price_output()  # вывод в формате:   цена за упаковку --- цена за миллилитр

['6000', '700', '306.000000', '306.05', Decimal('214200.000'), Decimal('0.00016340')]
['10000', '422', '505.890000', '505.89', Decimal('213485.580'), Decimal('0')]
['6000', '350', '637.440000', '727.62', Decimal('223104.000'), Decimal('0.14147214')]
['8000', '200', '521.290000', '521.29', Decimal('104258.000'), Decimal('0')]
['8000', '625', '740.850000', '1042.96', Decimal('463031.250'), Decimal('0.40778835')]
['8000', '313', '1060.73000', '1158.59', Decimal('332008.490'), Decimal('0.09225722')]
['6400', '390', '750.960000', '751.02', Decimal('292874.400'), Decimal('0.00007990')]
['10000', '2500', '739.750000', '858.4', Decimal('1849375.00'), Decimal('0.16039202')]
['6400', '3907', '513.360000', '585.37', Decimal('2005697.52'), Decimal('0.14027193')]
Цена госзакупки: 5698116.25
Цена контракта:  5698034.24
Разница:         82.01

1) 306.000000---0.051000
2) 505.890000---0.050589
3) 637.440000---0.106240
4) 521.290000---0.06516125
5) 740.850000---0.09260625
6) 1060.73000---0.13259125
7) 